LSTM

Import packages.

In [128]:
import torch
import os
import csv
import random
import numpy as np
import pandas as pd
import torch.nn.functional as F
import keras
import tensorflow as tf

from torch.nn.utils.rnn import pad_sequence
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

Download the dataset and unzip it.

In [129]:
!gdown 1vHDhZSPmhithLVRRzkNw0ak_kk7PhInu
!gdown 145T8z3XXlsaISzWdrJgJVANGokG1XuFI
!gdown 1uFCx21bqE3FnrdfvN_mwtw2-nvogTEex
!gdown 1zZo9RLt3mMmJZxEETSY2g9ND31qkZIn0
!gdown 1AubrUmeNUgpgiOu4tay6Gwl8O3lBaokF
!gdown 1ZOXGT_rIdEGIliHGKEH3ha77ZlZyq1Gn

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1vHDhZSPmhithLVRRzkNw0ak_kk7PhInu 

Downloading...
From: https://drive.google.com/uc?id=145T8z3XXlsaISzWdrJgJVANGokG1XuFI
To: /content/V_remit.csv
100% 90.4M/90.4M [00:00<00:00, 118MB/s]
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1uFCx21bqE3FnrdfvN_mwtw2-nvogTEex 

Downloading...
From: https://drive.google.com/uc?id=1zZo9RLt3mMmJZxEETSY2g9ND31qkZIn0
To: /content/V_cred.csv
100% 12.3M/12.3M [00:00<00:00, 179MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AubrUmeNUgpgiOu4tay6Gw

Basic setup of hyperparameters

In [142]:
'''
BATCH_SIZE = 256
EPOCH_NUM = 50
MAX_POSITIONS_LEN = 100
SEED = 97562246875 % (2**32-1) # Set your lucky number as the random seed
MODEL_DIR = 'model.pth'
lr = 5 * 1e-4

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

w2v_config = {'path': 'w2v.model', 'dim': 128}
net_config = {'hidden_dim': 64, 'num_layers': 3, 'bidirectional': False, 'fix_embedding': True}
header_config = {'dropout': 0.5, 'hidden_dim': 64}
assert header_config['hidden_dim'] == net_config['hidden_dim'] or header_config['hidden_dim'] == net_config['hidden_dim'] * 2
'''

Auxiliary functions and classes definition

In [151]:
V_cred = pd.read_csv("V_cred.csv", index_col=0) # 117: 13 months * 9 data
V_info = pd.read_csv("V_info.csv", index_col=0) # 4: risk_rank  occupation_code  total_asset  AGE             
V_remit = pd.read_csv("V_remit.csv", index_col=0) # 1572: 393 days * 4
train_Y = pd.read_csv("train_y.csv", index_col=0)

#print(V_cred)
#print(V_info)
#print(V_remit)
#print(train_X)
#print(train_y)
SAR_idx = []
for i in range (len(train_Y)):
    if (train_Y.iloc[i,0] == 1):
        SAR_idx.append(i)
print(len(SAR_idx), SAR_idx)

'''
# copy more data with SAR
# Maybe we can do this
for i in range (len(SAR_idx)):
    if (i % 50 == 0):
        print(i, "/", len(SAR_idx))
    for j in range (30):
        V_cred.loc[len(V_cred)] = V_remit.loc[SAR_idx[i]]
print(V_cred)
'''

234 [81, 181, 436, 1598, 1634, 1728, 1927, 2116, 2179, 2307, 2377, 2525, 2784, 2815, 2819, 2842, 2919, 2938, 2983, 3144, 3152, 3172, 3237, 3306, 3372, 3437, 3454, 3469, 3550, 3603, 3635, 3703, 3735, 3749, 3789, 3803, 3876, 3901, 3921, 3922, 3937, 4035, 4040, 4074, 4081, 4082, 4139, 4142, 4143, 4246, 4266, 4269, 4322, 4337, 4355, 4366, 4382, 4391, 4438, 4471, 4480, 4499, 4520, 4535, 4644, 4655, 4726, 4748, 4759, 4763, 4862, 4875, 4977, 5007, 5012, 5157, 5223, 5239, 5250, 5284, 5310, 5347, 5371, 5395, 5441, 5487, 5515, 5523, 5524, 5548, 5567, 5817, 5873, 5880, 5917, 5965, 6009, 6086, 6108, 6130, 6200, 6232, 6259, 6262, 6445, 6459, 6530, 6551, 6686, 6741, 6789, 6841, 6858, 6866, 6909, 6920, 6943, 6958, 6988, 7050, 7090, 7111, 7152, 7198, 7268, 7348, 7437, 7462, 7598, 7635, 7655, 7717, 7737, 7941, 7995, 8071, 8131, 8357, 8374, 8469, 8505, 8514, 8618, 8684, 8692, 8770, 8823, 9028, 9156, 9448, 9453, 9456, 9508, 9785, 9935, 10145, 10200, 10231, 10294, 10335, 10366, 10433, 10450, 10503, 10611,

'\n# copy more data with SAR\nfor i in range (len(SAR_idx)):\n    if (i % 50 == 0):\n        print(i, "/", len(SAR_idx))\n    for j in range (REPEAT_NUMBERS):\n        V_cred.loc[len(V_cred)] = V_remit.loc[SAR_idx[i]]\n\n\nprint(V_cred)\n'

In [154]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (13,9)))
model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.LSTM(units = 50, return_sequences = True))
#model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(units = 50))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='nadam')


tempx = np.array(V_cred)
tempy = np.array(train_Y)
print(tempx.shape)
tempx = np.reshape(tempx, (tempx.shape[0], 13, 9))
#tempy = np.reshape(tempy, (tempy.shape[0], tempy.shape[1], 1))
print(tempx.shape)
model.fit(tempx, tempy, epochs = 50, batch_size = 32) # epoch can be larger




(23906, 117)
(23906, 13, 9)
Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


748/748 [==============================] - 13s 12ms/step - loss: 0.0677
Epoch 2/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0550
Epoch 3/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0541
Epoch 4/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0544
Epoch 5/50
748/748 [==============================] - 10s 14ms/step - loss: 0.0542
Epoch 6/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0543
Epoch 7/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0540
Epoch 8/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0542
Epoch 9/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0535
Epoch 10/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0535
Epoch 11/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0535
Epoch 12/50
748/748 [==============================] - 9s 12ms/step - loss: 0.0536
Epoch 13/50
748/748 [=

In [155]:
test = tempx
#print(test)
'''
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) # Feature Scaling
'''
prediction = model.predict(test)
print(prediction, prediction.shape)
result = pd.concat([pd.DataFrame(prediction, columns=["prob"]), train_y], axis=1) 
result = result.sort_values(by = 'prob')
print(result)
for i in range (len(result)):
    if (result.iloc[i,1] == 1):
        print("The", i, "th lowest prediction has SAR with prob", result.iloc[i,0])

748/748 [==============================] - 3s 4ms/step
[[0.00922656]
 [0.00922656]
 [0.01416235]
 ...
 [0.01416235]
 [0.01416235]
 [0.00017959]] (23906, 1)
           prob  0
21558  0.000065  0
20359  0.000065  0
21421  0.000065  0
18366  0.000065  0
17445  0.000065  0
...         ... ..
7941   0.790488  1
5880   0.814637  1
3237   0.821980  1
6909   0.827619  1
3469   0.876941  1

[23906 rows x 2 columns]
The 4962 th lowest prediction has SAR with prob 0.0006045204
The 6334 th lowest prediction has SAR with prob 0.0021804383
The 7495 th lowest prediction has SAR with prob 0.0026288296
The 7794 th lowest prediction has SAR with prob 0.002628831
The 8197 th lowest prediction has SAR with prob 0.0028699054
The 8365 th lowest prediction has SAR with prob 0.0030627719
The 8720 th lowest prediction has SAR with prob 0.0036432114
The 8868 th lowest prediction has SAR with prob 0.0046093445
The 8900 th lowest prediction has SAR with prob 0.004807035
The 8911 th lowest prediction has SAR with 